In [41]:
import numpy as np
import pandas as pd
import optuna
from tqdm import tqdm
import sklearn
import tensorflow as tf
from tensorflow import keras
import scml
import mylib

In [42]:
scml.seed_everything()
pd.set_option("use_inf_as_na", True)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)

In [43]:
MODEL = 'efficientnetb3'
CONF = {
    'efficientnetb0': {
        'resolution': 224,
        'output_size': 1280,
    },
    'efficientnetb1': {
        'resolution': 240,
        'output_size': 0,
    },
    'efficientnetb2': {
        'resolution': 260,
        'output_size': 1408,
    },
    'efficientnetb3': {
        'resolution': 300,
        'output_size': 1536,
    },
    'efficientnetb4': {
        'resolution': 380,
        'output_size': 1792,
    },
    'efficientnetb5': {
        'resolution': 456,
        'output_size': 2048,
    },
    'efficientnetb6': {
        'resolution': 528,
        'output_size': 2304,
    },
    'efficientnetb7': {
        'resolution': 600,
        'output_size': 2560,
    },
}
INPUT_SHAPE = (CONF[MODEL]['resolution'], CONF[MODEL]['resolution'], 3)

In [44]:
train = pd.read_csv("input/train.csv", engine="c", low_memory=False)
train["target"] = mylib.target_label(train)
#train["image_path"] = "input/train_images/" + train["image"]
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34250 entries, 0 to 34249
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   posting_id   34250 non-null  object
 1   image        34250 non-null  object
 2   image_phash  34250 non-null  object
 3   title        34250 non-null  object
 4   label_group  34250 non-null  int64 
 5   target       34250 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.6+ MB


In [45]:
def _data_gen(
    dataframe,
    directory,
    target_size,
    batch_size,
    mode,
    color_mode="rgb",
    class_mode=None,
    x_col="image",
    y_col="label_group"
):
    rescale = 1./255
    shuffle = False
    idg = keras.preprocessing.image.ImageDataGenerator(rescale=rescale)
    if mode == "train":
        shuffle = True
        idg = keras.preprocessing.image.ImageDataGenerator(
            rescale=rescale,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True
        )
    return idg.flow_from_dataframe(
        dataframe=dataframe,
        x_col=x_col,
        y_col=y_col,
        directory=directory,
        target_size=target_size,
        color_mode=color_mode,
        batch_size=batch_size,
        shuffle=shuffle,
        class_mode=class_mode,
    )

In [46]:
spl = sklearn.model_selection.GroupKFold(n_splits=3)
x = train
y = train["label_group"]
groups = train["label_group"]
for ti, vi in spl.split(x, y, groups):
    val = train.iloc[vi]
    train_gen = _data_gen(
        dataframe=train.iloc[ti],
        directory="input/train_images",
        target_size=INPUT_SHAPE[:2],
        batch_size=32,
        mode="train"
    )
    val_gen = _data_gen(
        dataframe=train.iloc[vi],
        directory="input/train_images",
        target_size=INPUT_SHAPE[:2],
        batch_size=32,
        mode="val"
    )
    

Found 22832 validated image filenames.
Found 11418 validated image filenames.
Found 22834 validated image filenames.
Found 11416 validated image filenames.
Found 22834 validated image filenames.
Found 11416 validated image filenames.


In [ ]:
class MyObjective:
    def __init__(
        self,
        X,
        y,
        splitter,
        epochs: int,
        batch_size: int,
        patience: int,
        hidden_layer_sizes: List[int],
        job_dir: str,
        dropout: Tuple[float, float],
        lr: Tuple[float, float],
        groups=None,
    ):
        self.X = X
        self.y = y
        self.splitter = splitter
        self.epochs = epochs
        self.batch_size = batch_size
        self.patience = patience
        self.hidden_layer_sizes = hidden_layer_sizes
        self.job_dir = job_dir
        self.groups = groups
        self.dropout = dropout
        self.lr = lr
        self.history: List[Dict[str, Union[str, int, float]]] = []

    def __call__(self, trial):
        hist = {
            "trial_id": trial.number,
            "dropout": trial.suggest_uniform(
                "dropout", self.dropout[0], self.dropout[1]
            ),
            "learning_rate": trial.suggest_loguniform(
                "learning_rate", self.lr[0], self.lr[1]
            ),
        }
        scores = []
        for fold, (ti, vi) in enumerate(
            self.splitter.split(self.X, self.y, groups=self.groups)
        ):
            x_train, y_train = self.X[ti], self.y[ti]
            x_val, y_val = self.X[vi], self.y[vi]
            model = _model(
                input_dims=x_train.shape[1],
                dropout=hist["dropout"],
                lr=hist["learning_rate"],
                hidden_layer_sizes=self.hidden_layer_sizes,
            )
            model.summary()
            directory = f"{self.job_dir}/trial_{hist['trial_id']}/fold_{fold}"
            model.fit(
                x_train,
                y_train,
                epochs=self.epochs,
                batch_size=self.batch_size,
                validation_data=(x_val, y_val),
                callbacks=_callbacks(directory, es_patience=self.patience,),
            )
            y_pred = model.predict(x_val, batch_size=self.batch_size)
            score = metrics.mean_squared_error(y_val, y_pred, squared=False)
            log.info(f"score={score:.4f}, fold={fold}, trial={hist['trial_id']}")
            hist[f"fold_{fold}_score"] = score
            scores.append(score)
            del model
            gc.collect()
        hist["score_mean"] = np.mean(scores)
        hist["score_std"] = np.std(scores)
        hist["score_worst"] = max(scores)
        self.history.append(hist)
        return hist["score_worst"]

In [14]:
pretrained = keras.applications.EfficientNetB3(
    include_top=False,
    input_shape=INPUT_SHAPE,
    pooling="max",
    weights="pretrained/efficientnet/efficientnetb3_notop.h5"
)